# Библиотеки

In [24]:
from zlib import crc32

import numpy as np
import pandas as pd
import scipy.stats as st
from itertools import combinations
from statsmodels.stats.multitest import multipletests
import re

# Код генерации выборки

In [2]:
EMAIL_REGEX = re.compile(r"[^@]+@phystech.edu")

def generate_dataset(code):
    rs = np.random.RandomState(code)
    size = rs.randint(low=5, high=10)
    data = rs.randn(30, size)
    for i in range(size):
        data[:, i] += rs.uniform()*2-1.0
    return data

# Генерация выборки для вашей почты

<span style="color:red">
    ВАЖНО!
    Почта, которую укажете ниже и почта с которой Вы отправляете домашнее задание должна совпадать!
    В момент проверки задания алгоритм генерирует выборку на основе почты из анкеты!
</span>

Внимательно проверьте почту для которой выполняется задание!

In [3]:
task = dict()
task['mail'] = input(prompt='Enter your mail: ')
assert EMAIL_REGEX.match(task['mail']), 'Not a mail in the phystech.edu domain'
task['id'] = crc32(task['mail'].encode('utf-8'))
task['data'] = generate_dataset(task['id'])

task

Enter your mail: bogdanov.ai@phystech.edu


{'mail': 'bogdanov.ai@phystech.edu',
 'id': 1162440694,
 'data': array([[ 1.75750145e+00, -6.29772492e-01, -7.95937049e-01,
          5.14107649e-01,  2.71934149e-01,  2.45757706e-01],
        [-1.31745727e+00,  1.71582365e+00,  3.86234055e-01,
         -2.94497400e-01, -1.49196321e+00, -2.01468546e+00],
        [ 9.06885919e-01,  2.09119883e-01,  2.22987192e+00,
          1.89242468e+00,  4.23514654e-01, -4.57460779e-01],
        [ 2.22302813e+00, -1.62041271e+00,  1.20754551e+00,
         -5.82722395e-01,  2.26338741e+00,  6.98631501e-01],
        [-2.85882677e-01,  5.53456073e-02,  1.00669115e+00,
          1.86720509e+00,  8.71536359e-01, -4.88947907e-01],
        [ 5.43645169e-01, -4.22883163e-01, -1.48516156e-01,
         -8.52384555e-01, -1.79936191e+00, -4.35372859e-01],
        [ 2.26000978e+00, -1.33038322e+00,  1.01764257e+00,
         -2.59200218e-01,  2.37272533e+00, -4.88228617e-01],
        [ 3.80484416e-01, -2.36386376e-01,  1.91019303e+00,
          5.29333724e-01,  2.

# Работа с выборкой

In [8]:
data = task['data']

## Воспользуемся t-критерем для сравнения средних для всех пар столбцов из выборки. Для какого количества пар столбцов отвергается гипотеза о совпадении среднего?

In [26]:
def pairwise_ttest(data, alpha=0.05):

    cols = 0
    pvals = []
    data = pd.DataFrame(data)
    
    for col1, col2 in combinations(data.columns, 2):
        pvalue = st.ttest_ind(data[col1], data[col2])[1]
        pvals.append(pvalue)
        if pvalue <= 0.05:
            cols += 1
    return cols, pvals

print(f'Для {pairwise_ttest(data)[0]} пар')

Для 9 пар


## Значение уровня значимости при введении поправки Бонферрони (alpha=0.05). Округлить до 3 знака после запятой.

In [47]:
pvals = pairwise_ttest(data)[1]

p_new = multipletests(pvals, method='bonferroni')[-1]
print(f'Уровень значимости = {p_new:0.3f}')

Уровень значимости = 0.003


## Для какого количества пар столбцов отвергается гипотеза о совпадении среднего при использовании поправки Бонферрони?

In [28]:
pvals = pairwise_ttest(data)[1]

rejected = multipletests(pvals, method='bonferroni')[0].sum()
print(f'Отвергли для {rejected} пар столбцов')

Отвергли для 7 пар столбцов


## Для какого количества пар столбцов отвергается гипотеза о совпадении среднего при использовании метода Холма?

In [29]:
pvals = pairwise_ttest(data)[1]

rejected = multipletests(pvals, method='holm')[0].sum()
print(f'Отвергли для {rejected} пар столбцов')

Отвергли для 8 пар столбцов


## Оценка m0 по методу Шведера-Спьётволла (lambda = 0.5)

In [51]:
def shveder_spjetvoll(p_values, lmbd=0.5):
    return (1 + (np.array(p_values) > lmbd).sum()) / (1 - lmbd)

pvals = pairwise_ttest(data)[1]

print(f'mu_0 = {shveder_spjetvoll(pvals)}')

mu_0 = 4.0
